In [2]:
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain_core.runnables import RunnableLambda
from langchain_core.messages import HumanMessage
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# --- Agent definitions based on your CrewAI specs ---

def get_research_analyst():
    prompt = PromptTemplate(
        input_variables=["content"],
        template="""
You are a research analyst conducting an in-depth company research. Follow these instructions carefully:

{content}

Provide a detailed report including executive summary, company overview, leadership, services, technology stack, recent news, partnerships, strategic focus, opportunities, and verified citations.
"""
    )
    llm = OpenAI(temperature=0.3)
    return LLMChain(llm=llm, prompt=prompt)

def get_content_writer():
    prompt = PromptTemplate(
        input_variables=["content"],
        template="""
You are a content writer tasked with synthesizing research reports into a persuasive, concise, and structured B2B outreach summary.

{content}

Focus on key facts, strategic goals, technology gaps, suggested outreach angles, and a clear call to action.
"""
    )
    llm = OpenAI(temperature=0.5)
    return LLMChain(llm=llm, prompt=prompt)

def get_prospect_researcher():
    prompt = PromptTemplate(
        input_variables=["content"],
        template="""
You are a market researcher tasked with generating a list of companies likely to need software development services.

{content}

Output a Markdown table with company name, country, industry, funding stage, and a short rationale.
"""
    )
    llm = OpenAI(temperature=0.7)
    return LLMChain(llm=llm, prompt=prompt)

# Initialize agents
research_agent = get_research_analyst()
analysis_agent = get_content_writer()
prospect_agent = get_prospect_researcher()

# You can add tools here if needed
tools = []  # Example: [search_tool]

# State class (can be a simple dict)
class GraphState(dict):
    pass

# --- Define graph nodes ---

def research_company(state: GraphState) -> GraphState:
    company = state.get("company_name", "Unknown Company")
    prompt_content = f"""
Conduct an in-depth company research on: {company}

Steps:
1. Use reliable sources (official websites, news articles, LinkedIn, Crunchbase, etc.)
2. Identify key company details: overview, services, tech stack, leadership, financials, partnerships
3. Analyze strategic direction, market focus, recent developments, and potential software needs
4. Cross-verify facts across multiple sources
5. Document clearly with citations
"""
    result = research_agent.invoke([HumanMessage(content=prompt_content)])
    return {"research_report": result.content, **state}

def analyze_report(state: GraphState) -> GraphState:
    research_report = state.get("research_report", "")
    prompt_content = f"""
Based on the following research report, create a concise, persuasive summary for B2B outreach:

{research_report}

Focus:
- Executive summary (2–3 paragraphs)
- Key facts and strategic goals (bulleted)
- Tech gaps or improvement areas
- Suggested outreach/collaboration angle
- Call to action for marketing team

Make it clean, structured, and pitch-ready.
"""
    result = analysis_agent.invoke([HumanMessage(content=prompt_content)])
    return {"company_brief": result.content, **state}

def generate_prospect_list(state: GraphState) -> GraphState:
    prompt_content = """
Generate a list of at least 50 global companies likely to need software development services.

Focus sectors:
- Fintech, Healthtech, Edtech, Logistics, AI startups, Recently funded companies

For each company include:
- Name
- Country
- Industry
- Funding Stage (if known)
- One-line reason why they might need software development

Output as a Markdown table.
"""
    result = prospect_agent.invoke([HumanMessage(content=prompt_content)])
    return {"prospect_list": result.content, **state}

# --- Build LangGraph workflow ---

graph = StateGraph(GraphState)

graph.add_node("research", RunnableLambda(research_company))
graph.add_node("analyze", RunnableLambda(analyze_report))
graph.add_node("prospect", RunnableLambda(generate_prospect_list))

graph.set_entry_point("research")
graph.add_edge("research", "analyze")
graph.add_edge("analyze", END)

# Prospect runs in parallel with research/analyze
graph.add_parallel_edges("research", ["prospect"])
graph.add_edge("prospect", END)

compiled_graph = graph.compile()

# --- Run the workflow ---

initial_state = {
    "company_name": "Stripe"
}

result = compiled_graph.invoke(initial_state)

print("----- Company Brief -----\n")
print(result.get("company_brief", "No output"))
print("\n----- Prospect List -----\n")
print(result.get("prospect_list", "No output"))


ModuleNotFoundError: No module named 'langchain_community'